# Salazar
*A Python-writting wizard*

# I. Introduction

Coding, while rewarding and essential to modernity, can be monotonous at times. However, with modern Neural Networks, the possibilities exist for a world where we can start the initial stages of a programming process of a software solution and then allow a machine to finish the work for us. This concept entails a future where programmers could eliminate the overhead of debugging and testing and allow them more time to focus on the planning stage of project management. 

Generative text has been around for some time now but generative coding is still a relatively new implementation of it’s paradigms. The difficulties with generative code could be akin to training a model to write stories with a dataset containing mostly Sci-Fi and then expecting that model to write a Nicholas Sparks’ novel. I.e. The problem here stems from the multitude of libraries as packages used to build upon programming languages to make them useful for specific tasks. Just because a model can produce C code doesn’t mean it can build an operating system. So what do we do if we want a swiss army knife for coding nearly every variation of code in a specific language? Well maybe we should use a method that employs a significant amount of data (string of code) in tandem with a method of effectively seeding the model. There are, thankfully, enough similarities between any two programs written in Python that some rule should be learnable by a network; combining that with the right amount of “starter code” should prove effective enough to get relatively useful outputs.

## Goal

The goal is simple, albeit ambitious: develop a preprocessing method and LSTM model capable of generating source code from a supplied seed.

### Possible expansions in the future

Use an autencoder to generate a source code seed from English, from which the LSTM model can generate trailing code.


# II. Background
In the research paper "A deep language model for software code" researchers Hoa Khanh Dam, Truyen Tran, and Trang Pham; the minds behind Deepsoft cover their efforts in determining the validity of LSTMs in learning Java code [[1]](#References). Their argument for pursuing this model is based on the failure of models such as **n-grams** in capturing long context where dependent code elements scatter far apart. Dam and other also suggested that the unique capability of LSTMs in capturing "useful" patterns across a specified time step. In their research, the LSTM model performed well in generation Java code, so we decided this would be a good place to begin constructing our own model.

[To references](#References)

# III. Method
**Todo: add a breif decription to each list item**
### 1. Concatenate and Encode data
### 2. Reshape into sequneces
### 3. Training 
### 4. Testing

### Import Dependencies

In [184]:
# dependencies
"""
Numpy:             matrix manipulation and math
Pandas:            csv parsing and various data structure tasks
Mathpltlib.pyplot: data visualization
set_trace:         debug breaks
keras:             a machine learning library that is intuitive to read
tensorflow:        backend for keras; also the most widely used machine learning library
re:                regular expressions
pickle:            save objects to a file
"""
from copy import deepcopy as copy
from IPython.core.debugger import set_trace

import sys
import numpy as np
import pandas as pd
import scipy.special as sci
import matplotlib.pyplot as plt 
import os
import tensorflow as tf
import keras
import re
import pickle

tf.config.optimizer.set_jit(True) # optimizes cpu usage

from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline



## Concatenate files as a singular string
The bellow function uses a Python function knowns as ```walk``` to "walk" through a directory and read the files within that directory, storing them to the string ```content```. For our purposes, we have pulled from a github repository consisting of a large amount of Python code into a directory called `dataset` 

Currently using [this repository](https://github.com/TheAlgorithms/Python) as the dataset.

In [44]:
"""
concat_files
----------

Concatenate text files in a directory as a string

dependent on 'os' Python module

parameters
----------
directory: string; path of the target directory

f_type:    tuple of strings; target file extentsions Ex: ('.py', '.cpp')

return
---------
content:   concatenated string

"""
def concat_files(directory,f_type,excluded = ""):
    import os
    # List all file in the dataset directory
    # ------------------
    all_file = []
    content = ""

    # walk through every directory and open every f_type file
    # concatenate into var string "content"
    for root, dirs, files in os.walk(directory): 
        for name in files:
            if name.endswith(f_type) and not name.find(excluded): # we only care about .py
                all_file.append(name)
                with open(os.path.join(root,name), "r",encoding="utf8") as f:
                    content += f.read() + "\n"
    return content

In [61]:
content = concat_files("dataset",('.py'))
print(content[:100])

import math


def bisection(
    function, a, b
):  # finds where the function becomes 0 in [a,b] us


### Get a regular expression representation of ASCII characters
This will useful in distiguishing the characters that are important in terms of writting Python code vs. characters that are exclusive to documentation such as emoji's and other non-latin characters. This will help to slim our data shape, increasing training speed.

We learned later that there is a builtin python function, `chr()`, that can handle this with a bit more ease and is included in the final version of the function. 

In [4]:
r_all_ascii = "[^\x00-\x7F]"

## Bellow are two fuctions with the same name
[**The first**](#First-preprocessing-option) is for if you are *introducing new data to the dataset*, i.e. pulling new repositories into the "dataset" directory. This will perform the *encoding step*, converting every character in the concatenated string into an integer to be stored in a one dimensional array. 

[**The second**](#Second-preprocessing-option) is intended for loading an already encoded string and storing it into variable we will use going forward. This will save a signifigant amount of time, as it takes a while to encode a large string.

### First preprocessing option

In [156]:
"""
encode_string
-----------
Generate a dictionary representation of the characters found 
in a string keyed with integer representations

Returns two dictionaries and an array. The two dictionaries are 
necessary to convert the string to integer representation
and back again. The array is the string encoded as integer values.

parameters
----------
content:      string; to be processed

return
----------
vocab_to_int: dict; character to integer representation of unique characters in the string

int_to_vocab: dict; integer to string representation

encoded:      array; string encoded as integer values
"""

def pre_proc(content):   
    # Convert the string "content" into a list of intergers
#   -------------------------------------------------
#     ### creates a set of the individual characters
#     vocab = set(content)
#     ### attempt to clean out non-ascii characters
#     vocab_c = copy(vocab)
#     for i, char in enumerate(vocab_c):
#         if re.search(r_all_ascii,char):
#             vocab.remove(char)
#     print(vocab)
#     print(len(vocab))
#     ### use the set to sequentially generate a dictionary
#     vocab_to_int = {c: i for i, c in enumerate(vocab)} 
#     # print(vocab_to_int)
#     ### make keys the numerical values
#     int_to_vocab = dict(enumerate(vocab)) 
    
#     ### encode the "content" string using dict
#     ### encoded = np.array([vocab_to_int[c] for c in content], dtype=np.int32)
    
#     encoded = np.array([],dtype=np.int16)
#     for c in content:
#         if c in vocab_to_int:
#             encoded = np.append(encoded,vocab_to_int[c])
#   -------------------------------------------------


# use the bellow lines if you want a dictionary of all basic ASCII charcters.
# otherwise, comment out.
#   -------------------------------------------------
    comment = False
    quote_total = 0
    quote_comment = False
    string = False
    ap_string = False
    
    int_to_vocab = {i: chr(i) for i in range(127)}
    vocab_to_int = {chr(i): i for i in range(127)}

    encoded = np.array([],dtype=np.int16)
    for c in content:
        if c in vocab_to_int:
            if c == '"' and not (comment or ap_string):
                quote_total += 1
                if quote_total == 3:
                    quote_comment = not quote_comment
                if (quote_total == 1) and (not quote_comment):
                    string = not string
            elif c == "'" and not (comment or quote_comment or string):
                ap_string = not ap_string
            elif c == '#' and not (comment or quote_comment or string or ap_string):
                comment = True
            elif comment and c == '\n':
                comment = False
            if (not comment) and (not quote_comment):
                encoded = np.append(encoded,vocab_to_int[c]) 
            if c != '"':
                quote_total = 0
#             encoded = np.append(encoded,vocab_to_int[c])
                
                
    return vocab_to_int, int_to_vocab, encoded

### Second preprocessing option

In [107]:
# Run if you want to use data that is alread preprocessed 
def pre_proc(content):    
    import pickle
    
    infile1 = "./encodings/encoded2.txt"        # path to encoded string
    infile2 = "./encodings/vocab_to_int"
    infile3 = "./encodings/int_to_vocab"
    
    encoded = np.loadtxt(infile1, dtype=int) # load as an array of integers
    
#     with open(infile2, 'r') as fp:
#         vocab_to_int = pickle.load(fp)
    
#     with open(infile3, 'r') as fp:
#         int_to_vocab = pickle.load(fp)
        
    int_to_vocab = {i: chr(i) for i in range(127)}
    vocab_to_int = {chr(i): i for i in range(127)}
#   --------------------------------------------------    
    
    return vocab_to_int, int_to_vocab, encoded

### Run the preprocessing funtion
If you run the next cell, you will see the encoded string.

In [158]:
string = '# this is a test function \nfor i in range(5):\n   print("#this is source code#")\n"""\n   this is a \n   comment\n"""\nfor i in range(5):\n   print("this is more source code")'
vocab_to_int, int_to_vocab, test_encoded = pre_proc(string)
print ("\"", ''.join([int_to_vocab[value] for value in test_encoded]), "\"")

" 
for i in range(5):
   print("#this is source code#")
"""
for i in range(5):
   print("this is more source code") "


In [162]:
vocab_to_int, int_to_vocab, encoded = pre_proc(content)

In [163]:
#print(content)
print(int_to_vocab)
# this is all of the files concatenated. with each character encoded using the int_to_vocab
print()
print("Encoded string:",encoded)

{0: '\x00', 1: '\x01', 2: '\x02', 3: '\x03', 4: '\x04', 5: '\x05', 6: '\x06', 7: '\x07', 8: '\x08', 9: '\t', 10: '\n', 11: '\x0b', 12: '\x0c', 13: '\r', 14: '\x0e', 15: '\x0f', 16: '\x10', 17: '\x11', 18: '\x12', 19: '\x13', 20: '\x14', 21: '\x15', 22: '\x16', 23: '\x17', 24: '\x18', 25: '\x19', 26: '\x1a', 27: '\x1b', 28: '\x1c', 29: '\x1d', 30: '\x1e', 31: '\x1f', 32: ' ', 33: '!', 34: '"', 35: '#', 36: '$', 37: '%', 38: '&', 39: "'", 40: '(', 41: ')', 42: '*', 43: '+', 44: ',', 45: '-', 46: '.', 47: '/', 48: '0', 49: '1', 50: '2', 51: '3', 52: '4', 53: '5', 54: '6', 55: '7', 56: '8', 57: '9', 58: ':', 59: ';', 60: '<', 61: '=', 62: '>', 63: '?', 64: '@', 65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H', 73: 'I', 74: 'J', 75: 'K', 76: 'L', 77: 'M', 78: 'N', 79: 'O', 80: 'P', 81: 'Q', 82: 'R', 83: 'S', 84: 'T', 85: 'U', 86: 'V', 87: 'W', 88: 'X', 89: 'Y', 90: 'Z', 91: '[', 92: '\\', 93: ']', 94: '^', 95: '_', 96: '`', 97: 'a', 98: 'b', 99: 'c', 100: 'd', 101: 'e'

## $\rightarrow$ Save encoded array to avoid heavy computation

In [164]:
outfile1 = "./encodings/encoded2.txt"
outfile2 = "./encodings/vocab_to_int"
outfile3 = "./encodings/int_to_vocab"

np.savetxt(outfile1,encoded, fmt='%d')

with open(outfile2, 'wb') as fp:
    pickle.dump(vocab_to_int, fp)

with open(outfile3, 'wb') as fp:
    pickle.dump(int_to_vocab, fp)

## Reshape data into sequences

In [274]:
"""
sequenc_gen
---------------

Partition an array of encoded characters into sequences.

Parameters
---------------
encoded:         array of encoded characters; representation of a string
vocab_to_int:    dictionary for conversion from character to integer
int_to_vocab:    dictionary for conversion from integer to character

Settings
--------------
sequence_length: Specify the desired length of the sequences
"""
class Sequencer:
    def __init__(self,seq_len):
        self.seq_len = seq_len
        pass
    
    def sequence_gen(self, encoded, vocab_to_int, int_to_vocab):
        #return n_chars, n_vocab, n_patterns, datax, datay
        self.n_chars = len(encoded)
        self.n_vocab = len(vocab_to_int)
        self.datax = []
        self.datay = []

        # Loop through the encoded data and store 
        # sequences in datax and datay
        for i in range(0, self.n_chars - self.seq_len, 1):
            seq_in = encoded[i:i + self.seq_len] 
            seq_out = encoded[i + self.seq_len]
            self.datax.append(seq_in)
            self.datay.append(seq_out)
        self.n_patterns = len(self.datax)
        print("\nTotal patterns: ", self.n_patterns)
        print("Total unique characters: ", self.n_vocab)
        print ("\"", ''.join([int_to_vocab[value] for value in self.datax[100]]), "\"")

In [66]:
seq_len = 100
sequencer = Sequencer(100)
sequencer.sequence_gen(encoded,vocab_to_int, int_to_vocab)

Total patterns:  1022743
Total unique characters:  127
" = 0:  
        return a
    elif function(b) == 0:
        return b
    elif (
        function(a) * "


## Shape the sequences in a format that is better suited to LSTM cells
We will retain the pattern x sequence shape while adding a additonal dimension to represent the number of features; in this case one, since the each value can only represent one ASCII character. In terms of the LSTM, the sequence length will function as time steps for our LSTM cells.

In [12]:
def reshape_data(seq): 
    global X, Y
    from keras.utils import np_utils

    # reshape datax -- > [n_patterns, time steps, features]
    X = np.reshape(seq.datax, (seq.n_patterns,seq.seq_len,1))
    X = X / float(seq.n_vocab)
    Y = np_utils.to_categorical(seq.datay)
    #Y = np.asarray(datay) # for sparse categorical cross-entropy

In [13]:
reshape_data(sequencer)

## Model 
Here are the primary componets of the model built with Keras:
```python
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(Y.shape[1], activation='softmax')
```
Two layers in the model are commonly found in many other machine learning applications, so we will focus on the most detrimental layer(s) to the effectiveness of this model: the LSTM layer.

![](The_LSTM_cell.png)
*From wikipedia*

The image above depicts what is commonly found in **LSTM cells**. A typical LSTM has an additional input over vanilla RNNs known as the **cell-state vector**. This vector along with the **hidden-state vector** and **input data** allow the LSTM cell to "remember" or "forget" certain sequences [[1]](#References).

There are 3 primary gates within a cell that utilize the sigmoid function:
1. input gate $\rightarrow$ controls whether the memory cell is updated; contributing to the cell-state
2. forget gate $\rightarrow$ controls if the memory cell is reset to zero; also contributing to the cell-state
3. oupute gate $\rightarrow$ controls if the information of the current cell state is made visable; directly contributing to the hidden-state


**TODO: breifly explain compilation choice**
Because we are interested in generating source code based on a supplied seed, we want to predict the next character in a string based on the probably the next character being a specific choice in from our vocab, or dictonary of possible ASCII characters. Hence, the loss function we chose for our model is **categorical crossentropy**.
```python
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
```
We tried serveral different optimization options, but **adam** seemed to perform the best as far as producing a more accurate model. It also seems to the be the most commonly used optimizer in terms of text generation, as its adatptive learning rate excels in many applications.

## Compile Model
**TODO: maybe try different nn layers** More research needed.

In [289]:
def model_compile(**params):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Activation
    from tensorflow.keras.layers import Embedding
    from tensorflow.keras.layers import LSTM
    from tensorflow.keras.optimizers import RMSprop, SGD, Adam
    from tensorflow import keras
    from tensorflow.keras import layers
    # float16
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_policy(policy)


    global model
    
    dropout = params.pop("dropout",0.5)
    lstm_size = params.pop("lstm_size",128)
    secondL_scalar = params.pop("second_layer_scalar",1)
    
    model = Sequential()
    model.add(LSTM(lstm_size, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(LSTM(int(lstm_size * secondL_scalar),dropout = dropout))
    #model.add(Dropout(dropout))
    model.add(Dense(Y.shape[1], activation='softmax')) # output should be probabilities of character options

    #---------------------------------------
    # optimizer = RMSprop(learning_rate=0.05)

    # model.compile(loss='categorical_crossentropy',
    #               optimizer=optimizer,
    #               metrics=['accuracy'])
    #---------------------------------------
    opt = Adam(learning_rate = 0.001, beta_1 = 0.7875, beta_2 = 0.9999, epsilon = 9.166666666666666*pow(10,-9))

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

## Saving trained weights
Because this model takes a significant amount of time to train (about 10 minutes at 100 epochs on RTX 2070 super), we decided to save the weights that produce the lowest loss. The cell bellow will utillize keras' callback feature to save the current weights on each epoch that returns the new, lowest loss.

In [273]:
# checkpoint
import pickle
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
    
modelpath = "./models/best-weights.hdf5"
seqpath = "./models/sequencer"

# modelpath = "dummypath"
# seqpath = "dummypath2"

checkpoint = ModelCheckpoint(modelpath, monitor='loss',verbose=1, save_best_only=True, mode='min')

class SaveBest(Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.
       Format is based on tensorflow documentation 
    Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
    """
    
    def __init__(self, config, lowest_loss, index, validator=False, patience=0):
        super(SaveBest, self).__init__()

        self.patience = patience
        self.config = config
        self.lowest_loss = lowest_loss
        self.index = index
        self.validator = validator
        

        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None
        print("Initializing callback")

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        global best_seq, lowest_loss, configs
        
        current = logs.get('loss')
        self.config["loss"] = current
        
        if np.less(current, self.lowest_loss) and self.validator:
            print("\nNew best-weights found. Saving ...")
            best_seq = self.config["seq"] ## Need to determine best sequence length
            self.best = current
            self.lowest_loss = current
            lowest_loss = current ## Tracking lowest loss for future training
            self.wait = 0
            
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            
            self.model.save(modelpath)
            with open(seqpath, 'wb') as fp:
                pickle.dump(best_seq, fp)
        
        elif np.less(current, self.best) and not self.validator:
            self.best = current
            try:
                if np.less(self.best,configs[self.index]["loss"]):
                    configs[self.index] = self.config
                    print(self.config["loss"])
                    print("found new config")
                print("Checked loss",self.best,":",configs[self.index]["loss"])
            except:
                print("Initializing next config")
                configs.append(self.config)
#             configs[self.index] = self.config
            
            
#         else:
#             self.wait += 1
#             if self.wait >= self.patience:
# #                 self.stopped_epoch = epoch
#                 self.model.stop_training = True
#                 print('Shifting to next parameter')
# #                 self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))


In [291]:
best_seq = np.Inf

def parameter_search():
    global lowest_loss, configs 
    
    configpath = "./models/configs"
    
    lowest_loss = np.Inf
    configs = []
    
    sequence_lens = [10,20,30,40,50]
    dropouts = [0.2,0.3,0.4,0.5]
    lstm_sizes = [50,100,128]
    lstm_scalars = [0.5]
    epochs = [15] 
    
    for i,v in enumerate(lstm_sizes):
        for j,h in enumerate(lstm_scalars):
            for k,r in enumerate(dropouts):
                for p,e in enumerate(epochs):
                    for z,l in enumerate(sequence_lens):
                        sequencer = Sequencer(l)
                        sequencer.sequence_gen(encoded,vocab_to_int, int_to_vocab)
                        reshape_data(sequencer)
                        model_compile(dropout=r,lstm_size=v,second_layer_scalar=h)
                        print("fitting model with parameters:")
                        print("lstm size:",v)
                        print("lstm scalar:",h)
                        print("dropout:",r)
                        print("sequence length:",l)
                        config = {"seq":sequencer,"model":model,"epochs":e, "loss":np.inf}
                        try:
                            model.fit(X[:200000] , Y[:200000], epochs=e, batch_size=500, callbacks=[SaveBest(config,lowest_loss, z,False)])
                        except:
                            # TODO: write to error log
                            pass
    
    lowest_loss = np.inf
    epochs = 150
    for i, v in enumerate(configs):
        print(v)
        reshape_data(v["seq"])

        v["model"].fit(X[200000:400000] , Y[200000:400000], epochs=epochs, batch_size=500, callbacks=[SaveBest(v,lowest_loss, i,True)])

    for i, v in enumerate(configs):
        print(v)
        reshape_data(v["seq"])

        v["model"].evaluate(X[400000:600000] , Y[400000:600000], batch_size=500)
        

## Training
**TODO: explain training method once we get the best parameters**

We decided to take a parameter searching approach, as not much research is available to draw from when building a model that can write source code. So, based on text generation research, we started with a few basics parameters and kept the network somewhat shallow in order to stay within the RTX 2070's 8 GB of VRAM. We stepped away from **early stopping**, as was implemented by **Dam** and others [[2]](#References). we decided to compare all parameters with others immediatly by cross validating based on a global loss variable that stores the current loss value whenever a new best loss is discovered during training. Also, once a new-best-loss was discoverd, we saved both the model and the sequencer object to a separate files in order to use freely after training was complete.

### Parameter searching
Here we wrote a function to test several parameters. This function ran over the course of 3 days and proved to be sucessful in finding the best network configuration. Some of the values picked for the validation process were derived from Dam and others work, most noteably the mention that shorter sequence length seem to prevail in the case of generative code [[2]](#References).

- LSTM layer size $\rightarrow$ number of LSTM cells in the layer
- LSTM scalar $\rightarrow$ size of second LSTM layer based on the size of the first.
- Sequence length $\rightarrow$ number of characters to pass as input to the model
- Dropout rate (layer) $\rightarrow$ percentage of weights that are randomly ignored; combats overfitting
- Epochs $\rightarrow$ number of time data is passed into the model during the training process

### Execute parameter search

In [292]:
parameter_search()


Total patterns:  665198
Total unique characters:  127
" = 0:  
    "
fitting model with parameters:
lstm size: 50
lstm scalar: 0.5
dropout: 0.2
sequence length: 10
Initializing callback
Train on 200000 samples
Epoch 1/15
200000/200000 [==============================] - 9s 47us/sample - loss: 3.3281 - accuracy: 0.2828
Epoch 2/15
200000/200000 [==============================] - 3s 17us/sample - loss: 3.1976 - accuracy: 0.2885
Epoch 3/15
200000/200000 [==============================] - 3s 17us/sample - loss: 3.0604 - accuracy: 0.2888
Epoch 4/15
200000/200000 [==============================] - 3s 17us/sample - loss: 2.9340 - accuracy: 0.2916
Epoch 5/15
200000/200000 [==============================] - 3s 17us/sample - loss: 2.8630 - accuracy: 0.3000
Epoch 6/15
200000/200000 [==============================] - 3s 16us/sample - loss: 2.8072 - accuracy: 0.3044
Epoch 7/15
200000/200000 [==============================] - 3s 16us/sample - loss: 2.7707 - accuracy: 0.3069
Epoch 8/15
200000/200000 [

Epoch 2/15
200000/200000 [==============================] - 4s 20us/sample - loss: 3.1858 - accuracy: 0.2885
Epoch 3/15
200000/200000 [==============================] - 4s 20us/sample - loss: 3.0153 - accuracy: 0.2889
Epoch 4/15
200000/200000 [==============================] - 4s 20us/sample - loss: 2.9220 - accuracy: 0.2953
Epoch 5/15
200000/200000 [==============================] - 4s 20us/sample - loss: 2.8550 - accuracy: 0.3007
Epoch 6/15
200000/200000 [==============================] - 4s 19us/sample - loss: 2.8092 - accuracy: 0.3029
Epoch 7/15
200000/200000 [==============================] - 4s 19us/sample - loss: 2.7664 - accuracy: 0.3064
Epoch 8/15
200000/200000 [==============================] - 4s 19us/sample - loss: 2.7338 - accuracy: 0.3087
Epoch 9/15
200000/200000 [==============================] - 4s 19us/sample - loss: 2.7093 - accuracy: 0.3093
Epoch 10/15
200000/200000 [==============================] - 4s 19us/sample - loss: 2.6873 - accuracy: 0.3135
Epoch 11/15
200000

Epoch 4/15
200000/200000 [==============================] - 4s 22us/sample - loss: 2.9769 - accuracy: 0.2897
Epoch 5/15
200000/200000 [==============================] - 4s 22us/sample - loss: 2.9202 - accuracy: 0.2965
Epoch 6/15
200000/200000 [==============================] - 4s 22us/sample - loss: 2.8691 - accuracy: 0.3005
Epoch 7/15
200000/200000 [==============================] - 4s 22us/sample - loss: 2.8236 - accuracy: 0.3028
Epoch 8/15
200000/200000 [==============================] - 4s 22us/sample - loss: 2.7883 - accuracy: 0.3055
Epoch 9/15
200000/200000 [==============================] - 4s 22us/sample - loss: 2.7525 - accuracy: 0.3081
Epoch 10/15
200000/200000 [==============================] - 4s 22us/sample - loss: 2.7249 - accuracy: 0.3107
Epoch 11/15
200000/200000 [==============================] - 5s 23us/sample - loss: 2.7001 - accuracy: 0.3143
Epoch 12/15
200000/200000 [==============================] - 4s 22us/sample - loss: 2.6790 - accuracy: 0.3187
Epoch 13/15
2000

Epoch 6/15
200000/200000 [==============================] - 4s 18us/sample - loss: 2.8026 - accuracy: 0.3023
Epoch 7/15
200000/200000 [==============================] - 4s 18us/sample - loss: 2.7653 - accuracy: 0.3066
Epoch 8/15
200000/200000 [==============================] - 4s 18us/sample - loss: 2.7365 - accuracy: 0.3108
Epoch 9/15
200000/200000 [==============================] - 4s 19us/sample - loss: 2.7113 - accuracy: 0.3140
Epoch 10/15
200000/200000 [==============================] - 4s 19us/sample - loss: 2.6890 - accuracy: 0.3173
Epoch 11/15
200000/200000 [==============================] - 4s 18us/sample - loss: 2.6713 - accuracy: 0.3203
Epoch 12/15
200000/200000 [==============================] - 4s 18us/sample - loss: 2.6538 - accuracy: 0.3227
Epoch 13/15
200000/200000 [==============================] - 4s 18us/sample - loss: 2.6358 - accuracy: 0.3253
Epoch 14/15
198500/200000 [============================>.] - ETA: 0s - loss: 2.6189 - accuracy: 0.32802.6189111328125
found 

200000/200000 [==============================] - 4s 21us/sample - loss: 2.7555 - accuracy: 0.3082
Epoch 8/15
200000/200000 [==============================] - 4s 21us/sample - loss: 2.7267 - accuracy: 0.3107
Epoch 9/15
200000/200000 [==============================] - 4s 21us/sample - loss: 2.7024 - accuracy: 0.3151
Epoch 10/15
200000/200000 [==============================] - 4s 21us/sample - loss: 2.6826 - accuracy: 0.3192
Epoch 11/15
200000/200000 [==============================] - 4s 21us/sample - loss: 2.6654 - accuracy: 0.3222
Epoch 12/15
200000/200000 [==============================] - 4s 21us/sample - loss: 2.6486 - accuracy: 0.3260
Epoch 13/15
200000/200000 [==============================] - 4s 21us/sample - loss: 2.6333 - accuracy: 0.3281
Epoch 14/15
200000/200000 [==============================] - 4s 21us/sample - loss: 2.6155 - accuracy: 0.3319
Epoch 15/15
200000/200000 [==============================] - 4s 20us/sample - loss: 2.5985 - accuracy: 0.3356

Total patterns:  665158

...

200000/200000 [==============================] - 4s 22us/sample - loss: 1.6813 - accuracy: 0.5396
Epoch 140/150
200000/200000 [==============================] - 4s 22us/sample - loss: 1.6810 - accuracy: 0.5402
Epoch 141/150
200000/200000 [==============================] - 4s 22us/sample - loss: 1.6776 - accuracy: 0.5406
Epoch 142/150
200000/200000 [==============================] - 4s 22us/sample - loss: 1.6771 - accuracy: 0.5411
Epoch 143/150
200000/200000 [==============================] - 4s 22us/sample - loss: 1.6757 - accuracy: 0.5410
Epoch 144/150
200000/200000 [==============================] - 4s 22us/sample - loss: 1.6726 - accuracy: 0.5414
Epoch 145/150
200000/200000 [==============================] - 4s 22us/sample - loss: 1.6722 - accuracy: 0.5422
Epoch 146/150
200000/200000 [==============================] - 4s 22us/sample - loss: 1.6696 - accuracy: 0.5425
Epoch 147/150
200000/200000 [==============================] - 4s 22us/sample - loss: 1.6687 - accuracy: 0.5432
Epoch 

200000/200000 [==============================] - 5s 24us/sample - loss: 1.8592 - accuracy: 0.4933
Epoch 61/150
200000/200000 [==============================] - 5s 24us/sample - loss: 1.8553 - accuracy: 0.4943
Epoch 62/150
200000/200000 [==============================] - 5s 25us/sample - loss: 1.8521 - accuracy: 0.4955
Epoch 63/150
200000/200000 [==============================] - 5s 25us/sample - loss: 1.8469 - accuracy: 0.4967
Epoch 64/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.8428 - accuracy: 0.4976
Epoch 65/150
200000/200000 [==============================] - 5s 25us/sample - loss: 1.8438 - accuracy: 0.4976
Epoch 66/150
200000/200000 [==============================] - 5s 24us/sample - loss: 1.8393 - accuracy: 0.4990
Epoch 67/150
200000/200000 [==============================] - 5s 25us/sample - loss: 1.8357 - accuracy: 0.4996
Epoch 68/150
200000/200000 [==============================] - 5s 24us/sample - loss: 1.8385 - accuracy: 0.4992
Epoch 69/150
2

200000/200000 [==============================] - 5s 24us/sample - loss: 1.7077 - accuracy: 0.5302
Epoch 134/150
200000/200000 [==============================] - 5s 24us/sample - loss: 1.7064 - accuracy: 0.5297
Epoch 135/150
200000/200000 [==============================] - 5s 24us/sample - loss: 1.7021 - accuracy: 0.5314
Epoch 136/150
200000/200000 [==============================] - 5s 25us/sample - loss: 1.7063 - accuracy: 0.5302
Epoch 137/150
200000/200000 [==============================] - 5s 25us/sample - loss: 1.7010 - accuracy: 0.5310
Epoch 138/150
200000/200000 [==============================] - 5s 25us/sample - loss: 1.7009 - accuracy: 0.5319
Epoch 139/150
200000/200000 [==============================] - 5s 24us/sample - loss: 1.6985 - accuracy: 0.5314
Epoch 140/150
200000/200000 [==============================] - 5s 25us/sample - loss: 1.6970 - accuracy: 0.5325
Epoch 141/150
200000/200000 [==============================] - 5s 24us/sample - loss: 1.6988 - accuracy: 0.5318
Epoch 

200000/200000 [==============================] - 5s 27us/sample - loss: 1.8922 - accuracy: 0.4848
Epoch 55/150
200000/200000 [==============================] - 5s 27us/sample - loss: 1.8895 - accuracy: 0.4851
Epoch 56/150
200000/200000 [==============================] - 5s 27us/sample - loss: 1.8858 - accuracy: 0.4861
Epoch 57/150
200000/200000 [==============================] - 5s 27us/sample - loss: 1.8842 - accuracy: 0.4864
Epoch 58/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.8793 - accuracy: 0.4879
Epoch 59/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.8733 - accuracy: 0.4885
Epoch 60/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.8705 - accuracy: 0.4895
Epoch 61/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.8706 - accuracy: 0.4904
Epoch 62/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.8677 - accuracy: 0.4899
Epoch 63/150
2

200000/200000 [==============================] - 5s 26us/sample - loss: 1.7410 - accuracy: 0.5216
Epoch 128/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.7411 - accuracy: 0.5219
Epoch 129/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.7367 - accuracy: 0.5230
Epoch 130/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.7368 - accuracy: 0.5220
Epoch 131/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.7371 - accuracy: 0.5230
Epoch 132/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.7362 - accuracy: 0.5226
Epoch 133/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.7352 - accuracy: 0.5220
Epoch 134/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.7337 - accuracy: 0.5238
Epoch 135/150
200000/200000 [==============================] - 5s 26us/sample - loss: 1.7288 - accuracy: 0.5249
Epoch 

### Option1: Use documented best results (hardcoded from training results)

In [18]:
# import pickle
# testpath = "best-weights-test.hdf5"
# modelpath = testpath
# testpath2 = "sequencer-test"
# seqpath = testpath2

# best_seq = Sequencer(60)
# best_seq.sequence_gen(encoded,vocab_to_int, int_to_vocab)
# reshape_data(best_seq)

# # save sequencer object
# print(best_seq.seq_len)
# with open(seqpath, 'wb') as fp:
#     pickle.dump(best_seq, fp)
    
# model_compile(dropout=0.2,lstm_size=100,second_layer_scalar=1)
# # save model
# model.save(modelpath)

### Option2: or Load best weights recorded from training 

In [19]:
# from tensorflow.keras.models import load_model
# # filename should reflect the name of the best weights available 
# # in th local directory after training
# model.load_weights(modelpath)
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

In [280]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import RMSprop, SGD, Adam

modelpath = "./models/best-weights.hdf5"
seqpath = "./models/sequencer"

with open(seqpath, 'rb') as fp:
    best_seq = pickle.load(fp)

reshape_data(best_seq)

model = load_model(modelpath)

opt = Adam(learning_rate = 0.001, beta_1 = 0.7875, beta_2 = 0.9999, epsilon = 9.166666666666666*pow(10,-9))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [288]:
from keras.utils.vis_utils import plot_model
import h5py
h5py.run_tests()

# print(model.get_weights())

json_string = model.to_json()
print(best_seq.seq_len)

print(json_string)

Tests require pytest, pytest not installed
30
{"class_name": "Sequential", "config": {"name": "sequential_122", "layers": [{"class_name": "LSTM", "config": {"name": "lstm_242", "trainable": true, "batch_input_shape": [null, 30, 1], "dtype": {"class_name": "Policy", "config": {"name": "mixed_float16"}}, "return_sequences": true, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "time_major": false, "units": 128, "activation": "tanh", "recurrent_activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "recurrent_initializer": {"class_name": "Orthogonal", "config": {"gain": 1.0, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "unit_forget_bias": true, "kernel_regularizer": null, "recurrent_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "recurrent_constraint": null, "bias_constraint": null, "dropout": 0.0, "recu

### *(Optional)* train the model once again with new data and the best parameters

In [281]:
import datetime

try:
    model.fit(X[400000:600000] , Y[400000:600000], epochs=200, batch_size=500)
except:
    # TODO: write to error log
    pass

timestamp = datetime.datetime.now()

model.save(f"./models/refit.hdf5")

Train on 200000 samples
Epoch 1/200
200000/200000 [==============================] - 35s 177us/sample - loss: 2.2793 - accuracy: 0.4453
Epoch 2/200
200000/200000 [==============================] - 25s 123us/sample - loss: 2.0679 - accuracy: 0.4725
Epoch 3/200
200000/200000 [==============================] - 22s 112us/sample - loss: 1.9793 - accuracy: 0.4871
Epoch 4/200
200000/200000 [==============================] - 23s 114us/sample - loss: 1.9224 - accuracy: 0.4974
Epoch 5/200
200000/200000 [==============================] - 22s 112us/sample - loss: 1.8755 - accuracy: 0.5077
Epoch 6/200
200000/200000 [==============================] - 23s 115us/sample - loss: 1.8398 - accuracy: 0.5148
Epoch 7/200
200000/200000 [==============================] - 25s 123us/sample - loss: 1.8089 - accuracy: 0.5210
Epoch 8/200
200000/200000 [==============================] - 24s 121us/sample - loss: 1.7795 - accuracy: 0.5266
Epoch 9/200
200000/200000 [==============================] - 24s 121us/sample - 

200000/200000 [==============================] - 22s 110us/sample - loss: 1.2416 - accuracy: 0.6548
Epoch 74/200
200000/200000 [==============================] - 21s 107us/sample - loss: 1.2388 - accuracy: 0.6556
Epoch 75/200
200000/200000 [==============================] - 21s 107us/sample - loss: 1.2330 - accuracy: 0.6571
Epoch 76/200
200000/200000 [==============================] - 22s 108us/sample - loss: 1.2294 - accuracy: 0.6577
Epoch 77/200
200000/200000 [==============================] - 22s 109us/sample - loss: 1.2266 - accuracy: 0.6581
Epoch 78/200
200000/200000 [==============================] - 22s 108us/sample - loss: 1.2241 - accuracy: 0.6601
Epoch 79/200
200000/200000 [==============================] - 21s 107us/sample - loss: 1.2201 - accuracy: 0.6607
Epoch 80/200
200000/200000 [==============================] - 22s 108us/sample - loss: 1.2143 - accuracy: 0.6617
Epoch 81/200
200000/200000 [==============================] - 22s 109us/sample - loss: 1.2125 - accuracy: 0.6

200000/200000 [==============================] - 22s 108us/sample - loss: 1.0780 - accuracy: 0.6947
Epoch 146/200
200000/200000 [==============================] - 22s 109us/sample - loss: 1.0760 - accuracy: 0.6956
Epoch 147/200
200000/200000 [==============================] - 22s 108us/sample - loss: 1.0727 - accuracy: 0.6955
Epoch 148/200
200000/200000 [==============================] - 21s 106us/sample - loss: 1.0741 - accuracy: 0.6962
Epoch 149/200
200000/200000 [==============================] - 22s 108us/sample - loss: 1.0720 - accuracy: 0.6960
Epoch 150/200
200000/200000 [==============================] - 22s 108us/sample - loss: 1.0705 - accuracy: 0.6959
Epoch 151/200
200000/200000 [==============================] - 21s 107us/sample - loss: 1.0684 - accuracy: 0.6961
Epoch 152/200
200000/200000 [==============================] - 21s 106us/sample - loss: 1.0699 - accuracy: 0.6969
Epoch 153/200
200000/200000 [==============================] - 21s 107us/sample - loss: 1.0670 - accur

## Test model's ability to generate code
**TODO: try new prediction method**
This next cell uses a random sequence from the dataset as a seed to allow the model to predict the next characters. This will test the model's abiltiy to genrate code.
```python
prediction = model.predict(x, verbose=0)
index = sample(prediction)
result = int_to_vocab[index]
```
This portion of the code will use the seed to predict the next character. This is a basic operation in terms of classification so we will not go too deep into explaining this. The line, `index = sample(prediction)`, simply takes each vector of probabilities and indicates the index with the highest value, essential providing the predicted next-character. The next portion will append the predicted charcter to the seed pattern, repeating the process to predict the following character. `sample()` essentially reweights the probablity distrubution based on a certain temperature. This allows us to prevent any repetitive sequences from generating by introducing random character periodically.

The sampling fucntions is as detailed bellow and is based on a generative text inquiry [[3]](#References).


In [285]:
def sample(preds, temperature=0.1):
    # helper function to sample an index from a probability array
    # the higher the temperature the more likely the function
    # will pick a random character during the prediction process
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [287]:
# pick a random seed
start = np.random.randint(0, len(best_seq.datax)-1)
pattern = []
pattern = best_seq.datax[start]
print("Seed:")
print ("\"", ''.join([int_to_vocab[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(best_seq.n_vocab)
    prediction = model.predict(x, verbose=0)
    #index = np.argmax(prediction)
    index = sample(prediction[0])
    #index = np.random.choice(Y.shape[1], 1, p=prediction[0])[0]
    result = int_to_vocab[index]
    seq_in = [int_to_vocab[value] for value in pattern]
    sys.stdout.write(result)
    pattern = np.append(pattern, [index], axis=0) 
    length = pattern.shape[0] 
    pattern = pattern[1:length] 
print ("\nDone.")

Seed:
" ):
        menu.append(things( "
sat

C:\Users\Brandon\Anaconda3\envs\tf-gpu-cuda10\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


oex) int(str(indis))
            if len(selp, = 1 y <= 0:
        return self.nenslations] nor iene fint):,0 ondpe > [oi for l in rande(s)]


def sol_panesiat(natrix, fin)inputt):
    """

    # precondition
    assert isinstance(number, int) and (num_er1' nnd enltingtg fo tee latrix"
            "A. "Eneiartateo v methin the cilgnateoe of the post se enpcy )
        return self.rentletion(relf, lint):
        return self.reng

    def __terr__(self):
        return self.reng

    def __tt___(self, and int io ennation the sooted in the figgneng aor eisty and portide toer natrix dx {}i}irg forat )
        rrint("""- - Soaruare modph fs the ciriee seruetce fsr or a fiseramengry tu nest fineenate the palues cr {he cn {he postr fn {he pist so she cnrier c cor phe an ctemfatern: "
                        break
                    elif io % ind %n >= 0):
        return self.nensltialint

    def gant_dlerab(self, los) and itens < 0:
            return self.reng


def solution(n):
    """

Do

# References

1. [Greff, K., Srivastava, R. K., Koutník, J., Steunebrink, B. R., & Schmidhuber, J. (2016). LSTM: A search space odyssey. IEEE transactions on neural networks and learning systems, 28(10), 2222-2232.](https://arxiv.org/pdf/1503.04069.pdf?utm_content=buffereddc5&utm_medium=social&utm_source=plus.google.com&utm_campaign=buffer)
2. [Dam, H. K., Tran, T., & Pham, T. (2016). A deep language model for software code. arXiv preprint arXiv:1608.02715.](https://arxiv.org/pdf/1608.02715.pdf)
3. [Temperature based sampling to prevent repetitive generation](https://stackoverflow.com/questions/47125723/keras-lstm-for-text-generation-keeps-repeating-a-line-or-a-sequence)

[Back to top](#I.-Introduction)